# Output shuffles

1. Find all the segment addresses
2. For each segment, generate 32 inter-motif gap shuffles

In [1]:
from starling_rhythm.utils.paths import PROCESSED_DIR, ensure_dir
from starling_rhythm import synth

In [2]:
bID = 's_b1555_22'

In [3]:
bID_DIR = PROCESSED_DIR / bID
SEG_DIR = bID_DIR / 'segments'

In [4]:
SEG_DIR

PosixPath('/mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_b1555_22/segments')

In [5]:
seg_list = list(SEG_DIR.glob('*.wav'))

## choose randomly 500 of the segments

In [6]:
import random

In [7]:
## shuffle the list, and then pull out first 500
random.shuffle(seg_list)
seg_list = seg_list[:500]

## import labels

In [8]:
import pandas as pd

In [9]:
syllable_df = list(bID_DIR.glob('syllable_df_labels.pickle'))[0]
syllable_df = pd.read_pickle(syllable_df)

In [10]:
syllable_df

,file,onsets_ms,offsets_ms,umap,hdbscan_labels
0,/mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_...,0.754,1.826,"[-8.604912, 7.309538]",137
1,/mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_...,1.828,2.338,"[-10.148851, 5.063254]",112
2,/mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_...,3.680,4.537,"[-5.2042804, 12.267234]",189
3,/mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_...,5.632,6.140,"[-3.7597797, 11.497626]",-1
4,/mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_...,6.162,6.284,"[2.050827, 0.011981498]",193
...,...,...,...,...,...
60376,/mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_...,30.155,30.803,"[-11.164899, 2.6515484]",121
60377,/mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_...,30.905,31.609,"[3.2972312, -3.1319356]",146
60378,/mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_...,31.678,32.390,"[3.3780773, -3.0171862]",146
60379,/mnt/cube/j8xing/starling_rhythm_mfdfa/data/s_...,32.429,32.542,"[-6.3328857, 2.910542]",202


## Make test batch for the first address

In [11]:
batchsize = 32
tester = seg_list[0]

In [12]:
from starling_rhythm.synth import synth_song, save_synth, save_synth_shuffle_batch
import numpy as np

In [13]:
for n in np.arange(0, batchsize):
    shuffle_syn = save_synth(tester, syllable_df, bID_DIR, templating = False, gap_type = 'shuffle', n = n)

## Output all shuffles

In [14]:
from joblib import Parallel, delayed
from tqdm.autonotebook import tqdm

/tmp/ipykernel_3182/123448108.py:2: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [15]:
n_jobs = 47
verbose = 0

In [16]:
with Parallel(n_jobs = n_jobs, verbose = verbose) as parallel:
    parallel(
        delayed(save_synth_shuffle_batch)(
            wav_address,
            syllable_df,
            bID_DIR,
            templating = False,
            gap_type = 'shuffle',
            batchsize = batchsize
        )
        for wav_address in tqdm(seg_list, desc = 'Outputting 32 shuffles for each seg')
    )

Outputting 32 shuffles for each seg: 100%|██████████| 500/500 [03:36<00:00,  2.31it/s]
